In [ ]:
# https://towardsdatascience.com/pandas-groupby-a-simple-but-detailed-tutorial-314b8f37005d
# https://towardsdatascience.com/accessing-data-in-a-multiindex-dataframe-in-pandas-569e8767201d
# https://towardsdatascience.com/summarizing-data-with-pandas-crosstab-efc8b9abecf
# https://towardsdatascience.com/how-to-flatten-multiindex-columns-and-rows-in-pandas-f5406c50e569
# https://datascientyst.com/list-aggregation-functions-aggfunc-groupby-pandas/

In [3]:
import pandas as pd
import datetime
from myUtils import pickle_load, pickle_dump

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 30)
pd.set_option('display.max_colwidth', 18)
pd.set_option('display.width', 790)

path_dir = "C:/Users/ping/MyDrive/stocks/yfinance/"
path_data_dump = path_dir + "VSCode_dump/"

fp_df_eval_sym_freq_results_train = 'df_eval_sym_freq_results_train_2023-03-14'
fp_df_eval_sym_freq_results_validate = 'df_eval_sym_freq_results_validate_2023-03-14'
fp_df_eval_sym_freq_results_test = 'df_eval_sym_freq_results_test_2023-03-14'

str_today = datetime.date.today().strftime('%Y-%m-%d')
fp_results = f'yf_8_results_{str_today}'

In [ ]:
run_types = ['train', 'validate', 'test']

In [ ]:
for run_type in run_types:
  if run_type == 'train':
    df = pickle_load(path_data_dump, fp_df_eval_sym_freq_results_train)
    name_table = 'tbl_train'
  elif run_type == 'validate':
    df = pickle_load(path_data_dump, fp_df_eval_sym_freq_results_validate)
    name_table = 'tbl_val'
  elif run_type == 'test':
    df = pickle_load(path_data_dump, fp_df_eval_sym_freq_results_test)
    name_table = 'tbl_test'
  else:
    msg_stop = f"ERROR run_type must be 'train', 'validate', or 'test', run_type is: {run_type}"
    raise SystemExit(msg_stop )

  tbl = df.groupby(['days_lookbacks', 'days_eval', 'sym_freq_cnt'])\
          .agg({'grp(CAGR)_mean':          ['count', 'mean', 'std',],
                'grp(CAGR/UI)_mean':       ['mean', 'std',],
                'grp(CAGR/retnStd)_mean':  ['mean', 'std',],
                'SPY_CAGR':                ['mean', 'std',],
                'SPY_CAGR/UI':             ['mean', 'std',],
                'SPY_CAGR/retnStd':        ['mean', 'std',],                           
                })
  
  tbl['dif_CAGR_mean'] = tbl['grp(CAGR)_mean', 'mean'] - tbl['SPY_CAGR', 'mean']
  tbl['dif_CAGR/UI_mean'] = tbl['grp(CAGR/UI)_mean', 'mean'] - tbl['SPY_CAGR/UI', 'mean'] 
  tbl['dif_CAGR/retnStd_mean'] = tbl['grp(CAGR/retnStd)_mean', 'mean'] - tbl['SPY_CAGR/retnStd', 'mean'] 
  tbl['grp_CAGR/UI_mean/std'] = tbl['grp(CAGR/UI)_mean', 'mean'] / tbl['grp(CAGR/UI)_mean', 'std']  
  tbl['SPY_CAGR/UI_mean/std'] = tbl['SPY_CAGR/UI', 'mean'] / tbl['SPY_CAGR/UI', 'std'] 
  tbl['dif_CAGR/UI_mean/std'] = tbl['grp_CAGR/UI_mean/std']  - tbl['SPY_CAGR/UI_mean/std']
  tbl['pct_rank_d_CAGR/UI_m/s'] = tbl['dif_CAGR/UI_mean/std'].rank(pct=True)
  tbl['run_type'] = run_type
  
  pickle_dump(tbl, path_data_dump, name_table)
  print(f'saved table for run_type: {run_type} to {path_data_dump}{name_table}') 


In [ ]:
tbl_train = pickle_load(path_data_dump, 'tbl.train')
tbl_val = pickle_load(path_data_dump, 'tbl.validate')
tbl_test = pickle_load(path_data_dump, 'tbl.test')
print(f'len(tbl_train): {len(tbl_train)}')
print(f'len(tbl_val): {len(tbl_val)}')
print(f'len(tbl_test): {len(tbl_test)}')

In [ ]:
tbl_concat = pd.concat([tbl_train, tbl_val, tbl_test])
tbl_concat.sort_values(by=["days_lookbacks", "days_eval", "sym_freq_cnt"])
tbl_concat.head()

#### These are the best performing run parameters for days_lookbacks, days_eval, sym_freq_cnt. The performance metric is "pct_rank_d_CAGR/UI_m/s". The metric first calculate the difference between the group's "CAGR/UI_mean/CAGR/UI_std" and the SPY's "CAGR/UI_mean/CAGR/UI_std" for run_type "train", "validate" and "test". It does a percentile rank of the difference for each run_type. The hightest ranking is 1. For a set of run parameters, i.e. days_lookbacks, days_eval, sym_freq_cnt, the sum of "pct_rank_d_CAGR/UI_m/s" for the three run_types is 3 maximum.              

In [ ]:
yf_8_results = tbl_concat.groupby(level=[0,1,2]).sum(numeric_only=True).sort_values('pct_rank_d_CAGR/UI_m/s', ascending=False)
yf_8_results.head(8)

In [ ]:
pickle_dump(yf_8_results, path_data_dump, fp_results)
print(f'yf_8_results save to: {path_data_dump}{fp_results}')

In [4]:
df = pickle_load(path_data_dump, fp_results)
df.head()

grp(CAGR)_mean                             grp(CAGR/UI)_mean               grp(CAGR/retnStd)_mean                   SPY_CAGR                 SPY_CAGR/UI               SPY_CAGR/retnStd               dif_CAGR_mean dif_CAGR/UI_mean dif_CAGR/retnStd_mean grp_CAGR/UI_mean/std SPY_CAGR/UI_mean/std dif_CAGR/UI_mean/std pct_rank_d_CAGR/UI_m/s
                                                  count          mean           std              mean           std                   mean           std         mean           std          mean           std             mean           std                                                                                                                                           
days_lookbacks    days_eval sym_freq_cnt                                                                                                                                                                                                                                                                                                                                                 
[30, 60, 120]     4         8                       287  2.251341e+31  3.495021e+32      1.016651e+32  1.578266e+33       2.981200e+31      4.599171e+32  2506.100201  38499.178890  5.003596e+09  7.699365e+10     82666.934367  1.244125e+06  2.251341e+31     1.016651e+32       2.981200e+31              1.460210             0.712894             0.747316             2.674145    
[15, 30, 60, 120] 5         7                       454  2.210351e+54  3.966332e+55      4.942494e+60  8.868989e+61       5.038295e+54      9.040897e+55    19.609688    205.238892  1.312977e+07  1.872187e+08      1086.509738  7.030503e+03  2.210351e+54     4.942494e+60       5.038295e+54              1.109533             0.546658             0.562875             2.598932    
[30, 60, 120]     4         9                       159  1.406140e+05  9.798341e+05      2.674723e+11  1.958628e+12       8.857921e+06      6.661959e+07  4711.536608  52823.851676  9.405926e+09  1.056353e+11    153574.866921  1.706507e+06  1.359025e+05     2.580663e+11       8.704346e+06              1.524426             1.352416             0.172009             2.545726    
                            4                       561  3.010245e+28  4.251791e+29      2.688766e+30  3.797721e+31       6.552447e+28      9.254940e+29  1511.058721  29887.662537  3.016252e+09  5.977174e+10     50506.064853  9.683658e+05  3.010245e+28     2.688766e+30       6.552447e+28              0.799411             0.495010             0.304401             2.447436    
                  6         9                       168  1.573623e+04  1.665716e+05      5.524852e+08  6.108887e+09       2.041877e+05      1.964075e+06     7.046354     16.209956  2.477272e+05  2.547085e+06       893.586793  1.053718e+03  1.572918e+04     5.522375e+08       2.032941e+05              2.647089             2.278452             0.368637             2.414530

In [ ]:
# # select by days_lookbacks, days_eval, sym_freq_cnt
# tbl_concat.loc['[15, 30, 60, 120]', 4, 9]